<a href="https://colab.research.google.com/github/fofojaramillo/maestria-propedeuticos/blob/main/Proyecto_Estaditica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

La detección de fraudes es de vital importancia en el contexto de los pagos digitales, dado que el crecimiento de las transacciones electrónicas ha propiciado un aumento en los intentos de fraude perpetrados por ciberdelincuentes. Para garantizar la seguridad de las transacciones y proteger los fondos de las personas, numerosos bancos han implementado métodos avanzados de detección de fraudes y medidas de seguridad.

Este proyecto se enfoca en la aplicación de métodos estadísticos para la detección de fraudes en el ámbito de los pagos digitales. Mediante el análisis de datos y la identificación de indicadores relevantes, buscamos desarrollar un análisis estadístico inferencial que contribuya a proteger los sistemas financieros y a garantizar la integridad de las transacciones electrónicas.

# Problema

Teniendo la población de todas las transacciones hechas con una cuenta de banco. Tomamos una muestra aleatoria dentro de esta población de tamaño *n = 1000000* con los atributos (variables) enumeradas a continuación:

La base de datos utilizada consta de los siguientes variables:
- **distance_from_home**:  La distancia desde la casa en donde esta registrada la cuenta, hasta donde ocurrió la transacción.
- **distance_from_last_transaction**: La distancia desde la última transacción ocurrida, hasta esta transacción.
- **ratio_to_median_purchase_price**: Proporción del precio de compra de la transacción con respecto al precio medio de compras del usuario.
- **repeat_retailer**:  ¿Es la transacción realizada por el mismo comerciante?
- **used_chip**: ¿La transacción se realizó a través del chip de la tarjeta de crédito?
- **used_pin_number**: ¿La transacción se realizó utilizando un número PIN?
- **online_order**: ¿La transacción es un pedido en línea?
- **fraud**: ¿La transacción es fraudulenta?

El desafío principal radica en identificar los patrones y anomalías que indiquen posibles casos de fraude. Para abordar este problema, deberemos realizar un análisis exhaustivo de los datos y aplicar técnicas estadísticas.
A partir de estos datos se tratará de encontrar indicadores para detectar fraude.

# Objetivos
- Identificar los principales indicadores que generan un comportamiento atípico en los datos, lo que nos permitirá distinguir entre transacciones legítimas y fraudulentas
- Diseñar un una pequeña propuesta de supuestos que si se cumplen se tomen medidas precautorias para evitar el fraude.

# Analisis de datos



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cardTrans = pd.read_csv('/content/card_transdata.csv')

In [ ]:
cardTrans

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,1.0,1.0,0.0,0.0,0.0
999996,19.872726,2.683904,2.778303,1.0,1.0,0.0,0.0,0.0
999997,2.914857,1.472687,0.218075,1.0,1.0,0.0,1.0,0.0
999998,4.258729,0.242023,0.475822,1.0,0.0,0.0,1.0,0.0


In [ ]:
# Filtrar las transacciones fraudulentas y no fraudulentas
fraudulent_transactions = cardTrans.loc[cardTrans['fraud'] == 1.0]
non_fraudulent_transactions = cardTrans.loc[cardTrans['fraud'] == 0.0]

Se ha separado la base de datos en dos formas

## Leyendo la base de datos

## Exploración de la base de datos

## Limpieza de datos


## Estadística descriptiva


## Análisis estadístico inferencial


## Intervalos de confianza


## Resultados y conclusiones
